1. Get the Experience part text from the Image 

In [17]:
import os
from pdf2image import convert_from_path
import pytesseract

List the Resume

In [10]:
# Get the current directory
current_directory = os.getcwd()

# List directories in the current directory
resumes = ["datasets/resume/" + d for d in os.listdir(current_directory + "/datasets/resume")]
resumes

['datasets/resume/CV PUNITA SHAH.pdf',
 'datasets/resume/Manav Nirmal_Resume.pdf',
 'datasets/resume/Mokshit Jain Resume.pdf',
 'datasets/resume/Niraj_Resumev .pdf',
 'datasets/resume/Resume Siddharth Jain 1804.pdf',
 'datasets/resume/resume.pdf',
 "datasets/resume/Savita's Resume.pdf",
 'datasets/resume/VighneshChejara.pdf']

## using OCR read the text 

In [11]:
# Replace 'input_file.pdf' with the path to your PDF file
resume_pages = []

for pdf_file in resumes:
    pages = convert_from_path(pdf_file)

    resume_pages.append(pages)

In [18]:
def extract_text_from_image(image):
    text = pytesseract.image_to_string(image)
    return text

In [26]:
resumes_text = []

for pages in resume_pages:
    extracted_text = ""
    for page in pages:
        text = extract_text_from_image(page)
        extracted_text += text.replace("\n",' ') + "  " 
    resumes_text.append(extracted_text.strip())            

In [27]:
resumes_text

['PUNITA SHAH  TEAM LEAD (IT SUPPORT AND OPERATIONS)  CONTACTS  +91-97730 20235 shahpunita84@gmail.com  10, Navjivan Park, Near Thakorji Nagar, Civil Hospital Road, Nanakwada, Valsad 396 001, Gujarat, India  SKILLS  e Application Support e Production Support e JIRA  e AWS-CloudWatch e ServiceNow  e ActiveCollab  e Problem Solving  e@ Client Communication  LANGUAGES e English  e Guarati  e Hindi PERSONAL DETAILS  Date of birth 23/01/1984 Nationality  Indian  Marital status Single  ABOUT ME  5 Years of Experience of working as Team Lead of Application Support and Operatins Team  Interested in similar kind of Roles as well as seeking for any challenging opportunity in QA/BA or Data Analysis in reputed IT Firm  Ready to relocate anywhere in India.  WORK EXPERIENCE  Team Lead - Web Grabbing Support & Operations, Emsure Limited, SURAT Jan 2020 - Present  Client - Thomson Reuters, London  Roles and Responsibilities:  Responsible for End to End Support Services to the News & Editorial Bureau o

# Task 1 Extract the Experience part of each resume

In [32]:
experience_title = ['EXPERIENCE ', 'Experience ']

In [33]:
# extract the part of the Experience

experience_text = []

for resume_text in resumes_text:

    text = ''
    split_resume_text = resume_text.split(experience_title[0], 1)

    if len(split_resume_text) == 1:
        # try Experience
        split_resume_text = resume_text.split(experience_title[1], 1)

    # if the Experience is not found
    if len(split_resume_text) == 1 or len(split_resume_text[1].strip()) < 10:
        experience_text.append('No Experience Found')
        continue

    text = split_resume_text[1].strip()

    # end part Experince
    resume_title = ["EXTRA", "EDUCATION", "SKILLS", "LANGUAGES", "CERTIFICATIONS", "ACHIEVEMENTS", "CONTACT", "OBJECTIVE", "PERSONAL", "QUALIFICATIONS"
                    "ABOUT", "COURSE", "DETAILS", "PROJECTS", "INTERESTS"]

    for title in resume_title:
        text = text.split(title)[0]

    experience_text.append(text)



In [39]:
# check for people who don't have experience

for i in experience_text:
    print(i[:20])

Team Lead - Web Grab
Transaction Square L
CNK & ASSOCIATES LLP
htra & +91 993003233
Articleship firm - G
No Experience Found
Asp.net Developer Ga
No Experience Found


## Process the text 

In [42]:
import re
from datetime import datetime

current_year = datetime.now().year
current_month = datetime.now().month

In [84]:
def equi_space_text(text):

    # Split the text into words
    words = text.split()
    
    # Join the words with a single space between each word
    spaced_text = ' '.join(words)

    return spaced_text


In [94]:
def process_text(text, resume_name):

    total_year = 0
    total_months = 0
    
    text_array = text.split("-")

    for i in range(len(text_array) - 1):

        # Savita's Resume.pdf problem solve
        # CV PUNITA SHAH solve
        start_date = re.search(r'\b\d{2}/\d{4}\b', text_array[i].strip())
        end_date = re.search(r'\b\d{2}/\d{4}\b', text_array[i + 1].strip()) 

        if start_date and end_date:
            
            start_date = start_date.group()
            end_date = end_date.group()

            if len(text_array[i].strip().split(start_date)[1]) == 0 and len(text_array[i + 1].strip().split(end_date)[0]) == 0:
            
                start_month, start_year = map(int, start_date.split('/'))
                end_month, end_year = map(int, end_date.split('/'))
                
                # Convert start and end dates into datetime objects
                start_datetime = datetime(start_year, start_month, 1)
                end_datetime = datetime(end_year, end_month, 1)
                
                # Calculate the difference in years and months
                num_years = end_year - start_year
                num_months = end_month - start_month
                
                # Adjust for negative months
                if num_months < 0:
                    num_years -= 1
                    num_months += 12
                
                total_year += num_years
                total_months += num_months

    print(f"Total year of work experience of {resume_name} is ","{:.1f}".format(total_year + total_months/12))

In [67]:
def convert_date(text):

    date_strs = re.findall(r'(?:\b(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec) \d{4}\b|\b(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec) \d{4} - \d{2}/\d{4}\b)', text)

    for date_str in date_strs:
    
        # # Parse the date string
        date_obj = datetime.strptime(date_str, '%b %Y')
    
        # # Format the date as "MM/YYYY"
        formatted_date = date_obj.strftime('%m/%Y')

        text = text.replace(date_str, formatted_date)
        
    return text

In [75]:
def months_mapping(text):
    # Define a mapping of full month names to their abbreviations
    month_mapping = {
        'january': 'jan',
        'february': 'feb',
        'march': 'mar',
        'april': 'apr',
        'may': 'may',
        'june': 'jun',
        'july': 'jul',
        'august': 'aug',
        'september': 'sep',
        'october': 'oct',
        'november': 'nov',
        'december': 'dec'
    }
        
    # Compile regular expression pattern for full month names
    pattern = re.compile(r'\b(' + '|'.join(month_mapping.keys()) + r')\b', re.IGNORECASE)
    
    # Replace full month names with abbreviations
    text = pattern.sub(lambda x: month_mapping[x.group().lower()], text)
    
    return text

In [95]:

for i in range(len(experience_text)):

    text = experience_text[i]
    resume_name = resumes[i].split('datasets/resume/')[1] 
    
    new_text = text.lower().strip()
    
    current_month_str = str(current_month)
        
    if current_month < 10:
        current_month_str = "0" + str(current_month)
        
    new_text = new_text.replace('present', current_month_str + "/" + str(current_year))
    new_text = new_text.replace('till date', current_month_str + "/" + str(current_year))
    new_text = new_text.replace('to', '-')
    pattern = r'[^a-z0-9/+\-.@]'
        
    # Find all matches
    new_text = re.sub(pattern, ' ', new_text)
    
    new_text = equi_space_text(new_text)
    
    new_text = months_mapping(new_text)
    new_text = convert_date(new_text)

    process_text(new_text, resume_name)

Total year of work experience of CV PUNITA SHAH.pdf is  8.7
Total year of work experience of Manav Nirmal_Resume.pdf is  1.6
Total year of work experience of Mokshit Jain Resume.pdf is  0.7
Total year of work experience of Niraj_Resumev .pdf is  2.1
Total year of work experience of Resume Siddharth Jain 1804.pdf is  0.0
Total year of work experience of resume.pdf is  0.0
Total year of work experience of Savita's Resume.pdf is  1.0
Total year of work experience of VighneshChejara.pdf is  0.0


In [77]:
# Savita's Resume.pdf problem solve
# CV PUNITA SHAH solve
# Hitesh and Vignesh Problem solve
# Manav Nirmal resume problem solve